# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 23 2023 8:24AM,263449,19,TR-RNJ-05232023,"GCH Granules USA, Inc.",Released
1,May 23 2023 8:24AM,263448,20,REL1-6181,Reliable 1 Laboratories LLC,Released
2,May 23 2023 8:24AM,263448,20,REL1-6201,Reliable 1 Laboratories LLC,Released
3,May 23 2023 8:24AM,263448,20,REL1-6202,Reliable 1 Laboratories LLC,Released
4,May 23 2023 8:24AM,263448,20,REL1-6203,Reliable 1 Laboratories LLC,Released
5,May 23 2023 8:24AM,263448,20,REL1-6204,Reliable 1 Laboratories LLC,Released
6,May 23 2023 8:24AM,263448,20,REL1-6205,Reliable 1 Laboratories LLC,Released
7,May 23 2023 8:24AM,263448,20,REL1-6206,Reliable 1 Laboratories LLC,Released
8,May 23 2023 8:24AM,263448,20,REL1-6207,Reliable 1 Laboratories LLC,Released
9,May 23 2023 8:24AM,263448,20,REL1-6208,Reliable 1 Laboratories LLC,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
10,263444,Released,39
11,263445,Released,1
12,263447,Executing,1
13,263448,Released,23
14,263449,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
263444,NaN,NaN,39.0
263445,NaN,NaN,1.0
263447,NaN,1.0,NaN
263448,NaN,NaN,23.0
263449,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
263367,0.0,67.0,0.0
263369,0.0,1.0,0.0
263420,0.0,0.0,5.0
263422,5.0,0.0,6.0
263434,0.0,10.0,10.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
263367,0,67,0
263369,0,1,0
263420,0,0,5
263422,5,0,6
263434,0,10,10


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,263367,0,67,0
1,263369,0,1,0
2,263420,0,0,5
3,263422,5,0,6
4,263434,0,10,10


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,263367,,67,
1,263369,,1,
2,263420,,,5
3,263422,5,,6
4,263434,,10,10


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 23 2023 8:24AM,263449,19,"GCH Granules USA, Inc."
1,May 23 2023 8:24AM,263448,20,Reliable 1 Laboratories LLC
24,May 23 2023 8:14AM,263447,16,TASA USA Inc.
25,May 23 2023 8:11AM,263445,22,"NBTY Global, Inc."
26,May 23 2023 8:02AM,263444,19,"AdvaGen Pharma, Ltd"
65,May 23 2023 7:40AM,263443,19,"AdvaGen Pharma, Ltd"
95,May 22 2023 4:27PM,263441,10,ISDIN Corporation
96,May 22 2023 3:56PM,263439,19,"AdvaGen Pharma, Ltd"
119,May 22 2023 3:15PM,263434,20,Reliable 1 Laboratories LLC
139,May 22 2023 2:07PM,263422,10,Alikai Health


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 23 2023 8:24AM,263449,19,"GCH Granules USA, Inc.",,,1
1,May 23 2023 8:24AM,263448,20,Reliable 1 Laboratories LLC,,,23
2,May 23 2023 8:14AM,263447,16,TASA USA Inc.,,1,
3,May 23 2023 8:11AM,263445,22,"NBTY Global, Inc.",,,1
4,May 23 2023 8:02AM,263444,19,"AdvaGen Pharma, Ltd",,,39
5,May 23 2023 7:40AM,263443,19,"AdvaGen Pharma, Ltd",,,30
6,May 22 2023 4:27PM,263441,10,ISDIN Corporation,,1,
7,May 22 2023 3:56PM,263439,19,"AdvaGen Pharma, Ltd",,,23
8,May 22 2023 3:15PM,263434,20,Reliable 1 Laboratories LLC,,10,10
9,May 22 2023 2:07PM,263422,10,Alikai Health,5,,6


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2023 8:24AM,263449,19,"GCH Granules USA, Inc.",1,,
1,May 23 2023 8:24AM,263448,20,Reliable 1 Laboratories LLC,23,,
2,May 23 2023 8:14AM,263447,16,TASA USA Inc.,,1,
3,May 23 2023 8:11AM,263445,22,"NBTY Global, Inc.",1,,
4,May 23 2023 8:02AM,263444,19,"AdvaGen Pharma, Ltd",39,,
5,May 23 2023 7:40AM,263443,19,"AdvaGen Pharma, Ltd",30,,
6,May 22 2023 4:27PM,263441,10,ISDIN Corporation,,1,
7,May 22 2023 3:56PM,263439,19,"AdvaGen Pharma, Ltd",23,,
8,May 22 2023 3:15PM,263434,20,Reliable 1 Laboratories LLC,10,10,
9,May 22 2023 2:07PM,263422,10,Alikai Health,6,,5


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2023 8:24AM,263449,19,"GCH Granules USA, Inc.",1,,
1,May 23 2023 8:24AM,263448,20,Reliable 1 Laboratories LLC,23,,
2,May 23 2023 8:14AM,263447,16,TASA USA Inc.,,1,
3,May 23 2023 8:11AM,263445,22,"NBTY Global, Inc.",1,,
4,May 23 2023 8:02AM,263444,19,"AdvaGen Pharma, Ltd",39,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2023 8:24AM,263449,19,"GCH Granules USA, Inc.",1.0,NaN,NaN
1,May 23 2023 8:24AM,263448,20,Reliable 1 Laboratories LLC,23.0,NaN,NaN
2,May 23 2023 8:14AM,263447,16,TASA USA Inc.,NaN,1.0,NaN
3,May 23 2023 8:11AM,263445,22,"NBTY Global, Inc.",1.0,NaN,NaN
4,May 23 2023 8:02AM,263444,19,"AdvaGen Pharma, Ltd",39.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2023 8:24AM,263449,19,"GCH Granules USA, Inc.",1.0,0.0,0.0
1,May 23 2023 8:24AM,263448,20,Reliable 1 Laboratories LLC,23.0,0.0,0.0
2,May 23 2023 8:14AM,263447,16,TASA USA Inc.,0.0,1.0,0.0
3,May 23 2023 8:11AM,263445,22,"NBTY Global, Inc.",1.0,0.0,0.0
4,May 23 2023 8:02AM,263444,19,"AdvaGen Pharma, Ltd",39.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1053650,11.0,68.0,5.0
16,263447,0.0,1.0,0.0
19,1053775,93.0,0.0,0.0
20,526882,33.0,10.0,0.0
22,526814,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1053650,11.0,68.0,5.0
1,16,263447,0.0,1.0,0.0
2,19,1053775,93.0,0.0,0.0
3,20,526882,33.0,10.0,0.0
4,22,526814,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,11.0,68.0,5.0
1,16,0.0,1.0,0.0
2,19,93.0,0.0,0.0
3,20,33.0,10.0,0.0
4,22,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,11.0
1,16,Released,0.0
2,19,Released,93.0
3,20,Released,33.0
4,22,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,20,22
Status,,,,,
Completed,5.0,0.0,0.0,0.0,0.0
Executing,68.0,1.0,0.0,10.0,1.0
Released,11.0,0.0,93.0,33.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,20,22
0,Completed,5.0,0.0,0.0,0.0,0.0
1,Executing,68.0,1.0,0.0,10.0,1.0
2,Released,11.0,0.0,93.0,33.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,20,22
0,Completed,5.0,0.0,0.0,0.0,0.0
1,Executing,68.0,1.0,0.0,10.0,1.0
2,Released,11.0,0.0,93.0,33.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()